In [1]:
import os
import tensorflow as tf
import numpy as np
from scipy import stats
from image_data_module import TrainTestData
from salience_prediction_module import SaliencePrediction

os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [2]:
data_module = TrainTestData()
prediction_module = SaliencePrediction()

In [3]:
##### REGRESSION #####

for cross_validation_id in range(5):
    print("Evaluating split: CV_" + str(cross_validation_id))
    
    (X_train, Y_train), (X_test, Y_test) = data_module.get_train_test_salience(cv_name=str(cross_validation_id))
    (X_train, Y_train), (X_test, Y_test) = prediction_module.scale_data(X_train, Y_train, X_test, Y_test, labels='regression')
    print("Train:", X_train.shape, Y_train.shape, "Test:", X_test.shape, Y_test.shape)
    
    # data augmentation
    # X_train, Y_train = prediction_module.data_augmentation(X_train, Y_train, 2500) 
    
    vgg19 = prediction_module.initialize_cnn_for_regression(model_name='vgg19')
    
    # 25 epochs (128 batch size): ~16% mape, RMSprop 0.00002
    save_name = "vgg19_reg_" + str(cross_validation_id)
    prediction_module.train_cnn_for_regression(vgg19, X_train, Y_train, X_test, Y_test, epochs=25, batch_size=128,
                                               save=True, evaluate=True, save_name=save_name, verbose=0, plot=True)
    
    # 5 epochs (128 batch size): ~15% mape, RMSprop 0.000005
    vgg19 = prediction_module.fine_tuning(save_name)
    vgg19.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.000005),
                  loss='mean_absolute_error',
                  metrics=['mse', 'mae', 'mape'])
    
    save_name = "vgg19_reg_ft_" + str(cross_validation_id)
    save_name_model = "vgg19_reg_" + str(cross_validation_id)
    prediction_module.train_cnn_for_regression(vgg19, X_train, Y_train, X_test, Y_test, epochs=5, batch_size=128,
                                               save=True, evaluate=True, save_name=save_name_model, verbose=0)

Evaluating split: CV_0
Train: (1014, 298, 224, 3) (1014,) Test: (252, 298, 224, 3) (252,)
1014/1014 [==============================] - 3s 3ms/sample - loss: 0.0559 - mse: 0.0051 - mae: 0.0559 - mape: 11.3157
train loss, train mse, train mae, train mape: [0.05588447367239751, 0.0051251384, 0.055884477, 11.315692]
252/252 [==============================] - 1s 3ms/sample - loss: 0.0845 - mse: 0.0117 - mae: 0.0845 - mape: 16.3088
test loss, test mse, test mae, test mape: [0.08454020758942953, 0.0117166005, 0.08454021, 16.308762]
1014/1014 [==============================] - 3s 3ms/sample - loss: 0.0438 - mse: 0.0033 - mae: 0.0438 - mape: 8.7426
train loss, train mse, train mae, train mape: [0.043811005630201606, 0.003269626, 0.043811005, 8.742567]
252/252 [==============================] - 1s 3ms/sample - loss: 0.0855 - mse: 0.0116 - mae: 0.0855 - mape: 16.3960
test loss, test mse, test mae, test mape: [0.08548839520367366, 0.011550926, 0.085488394, 16.396015]
Evaluating split: CV_1
Train: 

<Figure size 432x288 with 0 Axes>

In [3]:
##### CLASSIFICATION #####

for cross_validation_id in range(0, 5):
    print("Evaluating split: cv_" + str(cross_validation_id))
    
    (X_train, Y_train), (X_test, Y_test) = data_module.get_train_test_binary(cv_name=str(cross_validation_id))
    (X_train, Y_train), (X_test, Y_test) = prediction_module.scale_data(X_train, Y_train, X_test, Y_test, labels='classification')
    print("Train:", X_train.shape, Y_train.shape, "Test:", X_test.shape, Y_test.shape)
    
    # data augmentation
    # X_train, Y_train = prediction_module.data_augmentation(X_train, Y_train, 2500)  
    
    class_weights = prediction_module.compute_class_weights(Y_train)
    print("Class weights:", class_weights)
    
    vgg19 = prediction_module.initialize_cnn_for_classification(model_name='vgg19', freeze=True, summary=False)
    
    # 35 epochs (128 batch size): ~0.65 acc, 0.63 loss, RMSprop 0.000005
    save_name = "vgg19_clf_" + str(cross_validation_id)
    prediction_module.train_cnn_for_classification(vgg19, X_train, Y_train, X_test, Y_test,
                                                   epochs=35, batch_size=128, class_weights=class_weights,
                                                   save=True, evaluate=True, save_name=save_name, verbose=0,
                                                   early_stopping=False, plot=True)
    
    # 5 epochs (128 batch size): ~ +0.02 acc, 0.63 loss, RMSprop 0.000001
    vgg19 = prediction_module.fine_tuning(save_name)
    vgg19.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.000001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    save_name = "vgg19_clf_ft_" + str(cross_validation_id)
    save_name_model = "vgg19_clf_" + str(cross_validation_id)
    prediction_module.train_cnn_for_classification(vgg19, X_train, Y_train, X_test, Y_test,
                                                   epochs=5, batch_size=128, class_weights=class_weights,
                                                   save=True, evaluate=True, save_name=save_name_model, verbose=0,
                                                   early_stopping=False)

Evaluating split: cv_0
Train: (1014, 298, 224, 3) (1014,) Test: (252, 298, 224, 3) (252,)
Class weights: {0: 1.005952380952381, 1: 0.9941176470588236}
  ...
    to  
  ['...']
  ...
    to  
  ['...']
1014/1014 [==============================] - 3s 3ms/sample - loss: 0.4514 - accuracy: 0.8531
train loss, train acc: [0.4514167312922092, 0.8530572]
252/252 [==============================] - 1s 3ms/sample - loss: 0.6557 - accuracy: 0.6349
test loss, test acc: [0.6556604495124211, 0.63492066]
  ...
    to  
  ['...']
  ...
    to  
  ['...']
1014/1014 [==============================] - 3s 3ms/sample - loss: 0.4137 - accuracy: 0.8698
train loss, train acc: [0.4136675914246185, 0.8698225]
252/252 [==============================] - 1s 3ms/sample - loss: 0.6537 - accuracy: 0.6508
test loss, test acc: [0.653710352996039, 0.6507937]
Evaluating split: cv_1
Train: (1014, 298, 224, 3) (1014,) Test: (252, 298, 224, 3) (252,)
Class weights: {0: 1.005952380952381, 1: 0.9941176470588236}
  ...
    to  

<Figure size 432x288 with 0 Axes>